In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

def initialize(zero_glue:tuple=(0,0), feed_rate:int=500, fast:bool=False) -> str:
    s='; Program initialization\n'
    s+=f'F{feed_rate}\n'
    s+='G00' if fast else 'G01'
    s+=f'X{zero_glue[0]:.3f}Y{zero_glue[1]:.3f}\n'
    s+='G91\n' # Relative coordinates
    s+='G00' if fast else 'G01'
    s+='; End of program initialization\n'
    return s

def moveTo(xy:tuple=(0,0), fast:bool=False, comment = '') -> str:
    x=xy[0]
    y=xy[1]
    s = 'G00' if fast else 'G01'
    if not x == 0:
        s+=f'X{x:.3f}'
    if not y == 0:
        s+=f'Y{y:.3f}'
    s+=comment
    s+='\n'
    return s

def glueDeposition(pause_long:float=1, pause_short:float=1, pause_glue:float=1) -> str:
    s = '; ------- Glue deposition -------\n'
    s+='M4\n' # Estensione asse Z
    s+=f'G4 P{pause_long:03d}\n' # Pausa per attesa estensione asse Z
    s+='M8\n' # Attivo il glue dispenser
    s+=f'G4 P{pause_short:03d}\n' # Pausa di un decimo del tempo di attesa standard
    s+='M9\n' # Disattivo il glue dispenser, in auto mode serve solo un impulso e lo simulo con la pausa sopra
    s+=f'G4 P{pause_glue:03d}\n' # Pausa per deposizione colla
    s+='M3\n' # Ritiro l'asse Z
    s+=f'G4 P{pause_long:03d}\n' # Pausa per attesa ritrazione asse Z
    s+='; ------- End of glue deposition -------\n'
    return s

In [9]:
glueDots = []
posX = True
posY = True
stepX = 20
stepY = 26.2
filename = 12 #Ladder lenght in number of silicons
glueDots_X = 51-((12-filename)*4)
for i in range(4):
    for j in range(glueDots_X):
        if i%2 == 0: # I numeri pari hanno step positivi, i dispari negativi
            glueDots.append((j*stepX, i*stepY))
        else:
            glueDots.append(((glueDots_X-j-1)*stepX, i*stepY))
glueCoord = np.array(glueDots)
#print(glueCoord)
oldXY = glueCoord[0]
print(len(glueDots))
with open(f'./{filename}Si.gcode', 'w') as gcode:
    gcode.write(initialize())
    for i in range(len(glueCoord)):
        xy = glueCoord[i]
        delta = (xy[0]-oldXY[0], xy[1]-oldXY[1])
        if not (delta[0] == 0 and delta[1] == 0):
            gcode.write(moveTo(xy=delta, comment=f' ; Glue dot in position ({i%51 if (i//51)%2 == 0 else (50-i%51)}, {(i//51)%4})'))
        else:
            gcode.write(f'; First glue dot in position ({i%51}, {(i//51)%4})\n')
        gcode.write(glueDeposition())
        oldXY = xy




204


In [11]:
# Print an image of the glue dots
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.scatter(glueCoord[:,0], glueCoord[:,1])
# Add a grid
ax.grid(True)
plt.show()
